In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
''' # Setting the hyper parameters of our model '''
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
# Read the file data and specifie delimiter
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
# For each line in the file we will extract input text and target text, they are delimited by a tab
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
# we will add each character in the input/ ouput character without repetition
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# No we will visualise the number of exemples we have
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


## Build the model

In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [6]:
'''
Now we will fit our model.
Before fitting our model, we choosed softmax as activation function for output layer and rmsprop as optimizer .
'''
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 35s 232ms/step - loss: 1.1405 - accuracy: 0.7366 - val_loss: 1.0103 - val_accuracy: 0.7205
Epoch 2/100
125/125 [==============================] - 24s 194ms/step - loss: 0.8147 - accuracy: 0.7794 - val_loss: 0.8392 - val_accuracy: 0.7697
Epoch 3/100
125/125 [==============================] - 24s 189ms/step - loss: 0.6610 - accuracy: 0.8162 - val_loss: 0.6955 - val_accuracy: 0.8013
Epoch 4/100
125/125 [==============================] - 24s 194ms/step - loss: 0.5711 - accuracy: 0.8342 - val_loss: 0.6353 - val_accuracy: 0.8144
Epoch 5/100
125/125 [==============================] - 25s 200ms/step - loss: 0.5217 - accuracy: 0.8474 - val_loss: 0.5817 - val_accuracy: 0.8308
Epoch 6/100
125/125 [==============================] - 26s 208ms/step - loss: 0.4858 - accuracy: 0.8571 - val_loss: 0.5495 - val_accuracy: 0.8407
Epoch 7/100
125/125 [==============================] - 26s 206ms/step - loss: 0.4587 - accuracy: 0.8638 - val_loss: 0.5306 -

Epoch 57/100
125/125 [==============================] - 29s 234ms/step - loss: 0.0950 - accuracy: 0.9698 - val_loss: 0.5709 - val_accuracy: 0.8766
Epoch 58/100
125/125 [==============================] - 29s 231ms/step - loss: 0.0927 - accuracy: 0.9704 - val_loss: 0.5797 - val_accuracy: 0.8753
Epoch 59/100
125/125 [==============================] - 30s 240ms/step - loss: 0.0907 - accuracy: 0.9711 - val_loss: 0.5799 - val_accuracy: 0.8757
Epoch 60/100
125/125 [==============================] - 33s 267ms/step - loss: 0.0886 - accuracy: 0.9715 - val_loss: 0.5841 - val_accuracy: 0.8759
Epoch 61/100
125/125 [==============================] - 32s 255ms/step - loss: 0.0869 - accuracy: 0.9723 - val_loss: 0.5828 - val_accuracy: 0.8761
Epoch 62/100
125/125 [==============================] - 27s 215ms/step - loss: 0.0847 - accuracy: 0.9730 - val_loss: 0.5945 - val_accuracy: 0.8753
Epoch 63/100
125/125 [==============================] - 28s 224ms/step - loss: 0.0834 - accuracy: 0.9731 - val_loss: 0

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


## Run inference (sampling)

In [7]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")
#encode input and retrieve initial decoder state

''' Encode input and retrieve initial decoder state '''
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

'''We will run one step of decoder with this initial state and a "start of sequence" token as target. 
Output will be the next target token.'''
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
'''we will repeat this step with the current target token and current states'''
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [8]:
'''In this step we will try to generate some decoded sentences'''
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Go.
Decoded sentence: En route !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Go.
Decoded sentence: En route !

1/1 [==============================] - 0s 34ms/step
-
Input sentence: Go.
Decoded sentence: En route !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Go.
Decoded sentence: En route !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 11ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 25ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Run!
Decoded sentence: Filez !

1/1 [==============================] - 0s 30ms/step
-
Input sentence: Run!
Dec

1/1 [==============================] - 0s 27ms/step
-
Input sentence: Run.
Decoded sentence: Filez !

1/1 [==============================] - 0s 16ms/step
-
Input sentence: Run.
Decoded sentence: Filez !

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Run.
Decoded sentence: Filez !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Run.
Decoded sentence: Filez !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Run.
Decoded sentence: Filez !

1/1 [==============================] - 0s 28ms/step
-
Input sentence: Run.
Decoded sentence: Filez !



In this exemple we build a simple character-level recurrent sequence-to-sequence model. We tried to translate short English senctences into French, sequence-to-sequence, that mean character by charater. With large text that is unusual because it can provide poor style of expression. The main idea is that an encoder LSTM turns input sequences to 2 state vectors and another decoder LSTM is trained to turn the target sequences into the same sequence but offset by one timestep in the future.